In [32]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from keras.layers import LSTM, Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [33]:
data = pd.read_csv(r"C:\Users\Jubin\Desktop\RP18\Datasets\intel\interpolated\injected_drift\mote=1_sensortype=temperature_faulttype=drift.csv")

In [34]:
sensor_data = data.drop(columns = ['timestamp', 'mote_id', 'light', 'has_fault_type'])
labels = data['has_fault_type']

In [35]:
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(labels)

In [36]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(sensor_data)

In [37]:
sequence_length = 100

X, Y = [], []
for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i : i + sequence_length])
    Y.append(Y_encoded[i + sequence_length])

X = np.array(X)
Y = np.array(Y)

In [38]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [39]:
# Step 2: Build the RNN Model
model = keras.Sequential([
    LSTM(units=128, return_sequences=True, input_shape=(sequence_length, 1)),
    BatchNormalization(),
    Dropout(0.3),
    LSTM(units=64, return_sequences=True),
    BatchNormalization(),
    Dropout(0.3),
    LSTM(units=32),
    BatchNormalization(),
    Dropout(0.3),
    Dense(units=1, activation='sigmoid')
])

In [40]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [41]:
# Step 3: Advanced Training Setup
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True, save_weights_only=False),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6)
]

In [42]:
# Step 4: Train the Model
history = model.fit(
    X_train, Y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_test, Y_test),
    callbacks=callbacks
)

Epoch 1/50


251/251 [==============================] - 73s 275ms/step - loss: 0.1089 - accuracy: 0.9799 - val_loss: 0.2618 - val_accuracy: 0.9260 - lr: 0.0010
Epoch 2/50
251/251 [==============================] - 72s 286ms/step - loss: 0.0319 - accuracy: 0.9962 - val_loss: 0.0315 - val_accuracy: 0.9958 - lr: 0.0010
Epoch 3/50
251/251 [==============================] - 73s 289ms/step - loss: 0.0331 - accuracy: 0.9958 - val_loss: 0.0263 - val_accuracy: 0.9963 - lr: 0.0010
Epoch 4/50
251/251 [==============================] - 73s 291ms/step - loss: 0.0172 - accuracy: 0.9972 - val_loss: 0.0304 - val_accuracy: 0.9950 - lr: 0.0010
Epoch 5/50
251/251 [==============================] - 74s 294ms/step - loss: 0.0182 - accuracy: 0.9974 - val_loss: 0.0276 - val_accuracy: 0.9970 - lr: 0.0010
Epoch 6/50
251/251 [==============================] - 74s 295ms/step - loss: 0.0149 - accuracy: 0.9980 - val_loss: 0.0210 - val_accuracy: 0.9980 - lr: 0.0010
Epoch 7/50
251/251 [==============================] - 74s 295ms

In [43]:
# Step 5: Evaluate the Model
model.load_weights('best_model.h5')  # Load the best model weights
Y_pred = model.predict(X_test)
Y_pred_binary = (Y_pred > 0.5).astype(int)

126/126 [==============================] - 8s 54ms/step


In [44]:
accuracy = accuracy_score(Y_test, Y_pred_binary)
precision = precision_score(Y_test, Y_pred_binary)
recall = recall_score(Y_test, Y_pred_binary)
f1 = f1_score(Y_test, Y_pred_binary)
conf_matrix = confusion_matrix(Y_test, Y_pred_binary)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.9982552342971087
Precision: 0.9941434846266471
Recall: 0.9956011730205279
F1 Score: 0.9948717948717949
Confusion Matrix:
[[3326    4]
 [   3  679]]
